In [1]:
#Loading packages

In [43]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from __future__ import print_function
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.io as io

In [3]:
#Loading SDF file of ligands on Pandas

In [4]:
SDF = '/home/daniele/Desktop/plip/logistic_regression/ligands.sdf'

In [5]:
dataframe_ligands = PandasTools.LoadSDF(SDF)

In [6]:
#Selecting SMILES from SDF file

In [7]:
name_smiles_ligands=dataframe_ligands[['ChemCompId', 'SMILES']]

In [8]:
set_smiles = name_smiles_ligands.drop_duplicates()

In [9]:
set_smiles.columns = ['LIG_PDB', 'SMILES']

In [10]:
#Mapping PDB IDs to ChEMBL

In [11]:
pdb_to_chembl=pd.read_csv('/home/daniele/Desktop/plip/map/src1src3.txt' , sep='\t', usecols=['LIG_CHEMBL','LIG_PDB'], dtype= {'LIG_CHEMBL': np.str, 'LIG_PDB': np.str} )

In [12]:
smiles_mapped = pd.merge(set_smiles, pdb_to_chembl, on= [ 'LIG_PDB'])

In [13]:
#Filtering smile list for PLIP ligands in ChEMBL IDs

In [14]:
ic50_from_plip = pd.read_csv('/home/daniele/Desktop/plip/sparseflow/ic50_from_plip.txt')

In [15]:
c = pd.DataFrame()

In [16]:
c = ic50_from_plip[['0','1']]

In [17]:
c.columns = ['LIG_CHEMBL', '1']

In [18]:
smiles_plip=pd.merge(smiles_mapped, c, on= [ 'LIG_CHEMBL'])

In [19]:
chembl_smiles = smiles_plip[['LIG_CHEMBL', 'SMILES']]

In [20]:
#ECFP generation

In [21]:
a= []
b= []
for mol in chembl_smiles['SMILES']:
    m = Chem.MolFromSmiles(mol)
    info={}
    AllChem.GetMorganFingerprint(m,3,bitInfo=info)
    a += [mol, info]
    b += [a]
    a= []
data_ecfp = pd.DataFrame(b,columns=['SMILES','ECFP'])

In [22]:
names_ecfp_merged = pd.merge(chembl_smiles, data_ecfp, on= [ 'SMILES'])

In [23]:
#ECFP features selection

In [24]:
list_of_feat = []
for compound in names_ecfp_merged['ECFP']:
    for key in compound:
        list_of_feat += [key]
set_list_of_feat = set(list_of_feat)

In [25]:
list_feat = []
for feat in set_list_of_feat:
    list_feat += [feat]
list_feat.sort()

In [26]:
a = []
b = []
for compound in names_ecfp_merged['ECFP']:
    for key in compound:
        a += [key]
    b += [a]
    a = []

In [27]:
names_ecfp_merged['FEAT']=pd.Series(b)

In [28]:
#Binary fingerprint generation

In [29]:
s = []

for compound in names_ecfp_merged['FEAT']:
    fingerprint=[0] * 43151
    for f in list_feat:
        if f in compound:
            i = list_feat.index(f)
            fingerprint[i-1] = 1
    s += [fingerprint]

In [30]:
names_ecfp_merged['FINGERPRINT']=pd.Series(s)

In [31]:
#X matrix preparation

In [32]:
lig_fing = names_ecfp_merged[['LIG_CHEMBL', 'FINGERPRINT']]

In [33]:
cou = 0
for f in lig_fing['FINGERPRINT'][0]:
    cou += 1

In [34]:
cou = range(0,43151)
for cc in cou:
    lig_fing[cc] = 0

/home/daniele/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [35]:
lig_fing_sorted = lig_fing.sort_values(by=['LIG_CHEMBL'])

In [36]:
lig_fing_sorted.reset_index(inplace=True)

In [37]:
mylist = lig_fing_sorted['FINGERPRINT'][0]
def indices( mylist, value):
    return [i for i,x in enumerate(mylist) if x==value]

In [38]:
ecfp = pd.DataFrame()

In [39]:
ecfp = lig_fing_sorted.iloc[:, 3:]

In [ ]:
lig_fing_sorted['LIG_CHEMBL']

In [40]:
le = range(2597)
for l in le:
    ind = indices(lig_fing_sorted['FINGERPRINT'][l], 1)
    for i in ind:
        ecfp[i][l] = 1

In [41]:
ecfp.values.shape

(2597, 43151)

In [ ]:
ecfp.to_csv('/home/daniele/Desktop/plip/sparseflow/ecfp.txt')

In [44]:
x = sp.coo_matrix(ecfp.values)

In [ ]:
io.mmwrite('/home/daniele/Desktop/plip/sparseflow/ecfp_sparse.mtx', x)